## Imports

In [22]:
import numpy as np
import pandas as pd
import re
import torch
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
torch.cuda.is_available()
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

## Device Configuration

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Mount Drive

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Get author name

In [25]:
from google.colab import auth
import google.auth
from googleapiclient.discovery import build

auth.authenticate_user()
creds, _ = google.auth.default(scopes=['https://www.googleapis.com/auth/drive'])

drive_service = build('drive', 'v3', credentials=creds)
about = drive_service.about().get(fields="user").execute()
user_name = about['user']['displayName']


## Text preprocessing

In [26]:
stopChars = [',','(',')','.','-','[',']','"', '{', '}']

def preprocessText(text):
    processedText = text.lower()
    processedText = re.sub(r'[^a-zA-Z\s\.,;!?"\'\[\]]', '', processedText)
    processedText = re.sub(r'\([^()]*\)|\[[^\[\]]*\]|\{[^{}]*\}', '', processedText)
    for char in stopChars:
        processedText = processedText.replace(char,'')

    return processedText

## Models Definition

In [27]:
class Word_LSTM(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, num_genres=2, genre_embedding_dim=128, dropout=0.4):
        super(Word_LSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.genre_embedding_dim = genre_embedding_dim
        self.genre_embedding = nn.Embedding(num_genres, genre_embedding_dim)
        self.word_embeddings = nn.Embedding(n_vocab, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim + genre_embedding_dim, hidden_dim, dropout=dropout, num_layers=2)
        self.fc = nn.Linear(hidden_dim, n_vocab)

    def forward(self, seq_in, genre_in):
        # Embedding layer for words
        embedded_words = self.word_embeddings(seq_in.t())

        # Embedding layer for genre
        genre_embedded = self.genre_embedding(genre_in.t())


        # Concatenate word embeddings with genre embeddings along the feature dimension
        concatenated = torch.cat((embedded_words, genre_embedded), dim=2)

        # LSTM layer
        lstm_out, _ = self.lstm(concatenated)

        # Last hidden state
        ht = lstm_out[-1]

        # Fully connected layer
        out = self.fc(ht)

        return out

## Load The Model

In [28]:
# Load rap model and dictionaries
checkpoint = torch.load('/content/drive/MyDrive/Final_Project/Models/model.pt')
word_to_int = checkpoint['word_to_int']
int_to_word= checkpoint['int_to_word']

model = Word_LSTM(90498, 256, 256).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

Word_LSTM(
  (genre_embedding): Embedding(2, 128)
  (word_embeddings): Embedding(90498, 256)
  (lstm): LSTM(384, 256, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=256, out_features=90498, bias=True)
)

## Sample the predictions

In [29]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    EPSILON = 1e-9
    preds = np.log(preds + EPSILON) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Generation function

In [30]:
def generate_song_with_genre(model, int_to_word, word_to_int, starting_sentence, genre, seq_length=10, variance=0.8, max_length=400):
    generated = []
    window = starting_sentence
    newline_count = 0  # Variable to count consecutive newline characters

    for i in range(max_length):
        x = np.zeros((1, seq_length))
        g = np.zeros((1, seq_length))
        for t, word in enumerate(window):
            x[0, t] = word_to_int.get(word, -1)  # Check if word is in vocabulary
            g[0, t] = genre
        if -1 in x:
            print(f"Warning: The word '{window[x[0].tolist().index(-1)]}' is not in the vocabulary!")

        x_in = torch.LongTensor(x)
        genre_in = torch.LongTensor(g)
        pred = model(x_in, genre_in)
        pred = np.array(F.softmax(pred, dim=1).data[0].cpu())
        next_index = sample(pred, variance)
        next_word = int_to_word.get(next_index, '<unk>')  # Convert index to word

        if next_word == "\n":
          newline_count += 1
          if newline_count > 2:
              continue  # Skip adding more newline characters if already two consecutive
        else:
            newline_count = 0  # Reset newline count for new word

        generated.append(next_word)
        window = window[1:] + [next_word]

    return starting_sentence + generated


## Encode Genere

In [31]:
def encode_genre(genre):
    return 0 if genre == 'rap' else 1  # Encode 'rap' as 0 and 'pop' as 1

## Initate Generation

In [33]:
while True:
    genre = input("Enter the genre (rap/pop): ").lower()

    if genre not in ['rap', 'pop']:
        print("Invalid genre! Please enter 'rap' or 'pop'.")
        continue

    while True:
        starting_sentence = input("Enter the starting sentence with exactly 10 words: ").lower().split()

        if len(starting_sentence) != 10:
            print("Invalid starting sentence! Please enter exactly 10 words.")
            continue

        starting_sentence = [preprocessText(word) for word in starting_sentence]
        break

    # Once the inputs are valid, generate the song lyrics
    try:
        genre = encode_genre(genre)
        song_lyrics = generate_song_with_genre(model, int_to_word, word_to_int, starting_sentence, genre)
        while True:
            user_file_name = input("Enter your song name: ").strip()
            if user_file_name == '':

                print("Please enter a valid file name.")
                continue

            file_path = f'/content/drive/MyDrive/Final_Project/Outputs/{user_file_name}.txt'
            try:
                # Save generated song lyrics to file
                with open(file_path, 'w') as file:
                    file.write(f"{user_file_name}, by {user_name}\n\n")
                    file.write(" ".join(song_lyrics))

                print(f"Generated song lyrics saved to '{file_path}'.")
                break  # Exit the loop if the song is successfully saved
            except OSError as e:
                print(f"Error: {e.strerror}. Please enter a different file name.")

        break  # Exit the loop if the song generation is successful
    except KeyError as e:
        print(f"Error: Word '{e.args[0]}' not found in the vocabulary. Please enter another starting sentence.")


Enter the genre (rap/pop): rap
Enter the starting sentence with exactly 10 words: every morning the sun rises and every night the moon
Enter your song name: new_rap_song
Generated song lyrics saved to '/content/drive/MyDrive/Final_Project/Outputs/new_rap_song.txt'.
